In [11]:
import cv2
import numpy as np

# Load the video
cap = cv2.VideoCapture(r"C:\Users\ACER\Desktop\OPEN_CV\119278-717347141_small.mp4")

# Read the first frame
ret, first_frame = cap.read()

# Check if the first frame was successfully read
if not ret:
    print("Failed to read the first frame")
    cap.release()
    exit()

# Resize the first frame to a medium screen size (640x360)
first_frame = cv2.resize(first_frame, (640, 360))

# Select ROI (Region of Interest) for the object to be tracked
roi = cv2.selectROI("Select ROI", first_frame, False)
x, y, w, h = roi
template = first_frame[y:y+h, x:x+w]

# Convert the template to grayscale
template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

# Define the method for template matching
method = cv2.TM_CCOEFF_NORMED

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame to the same medium screen size
    frame_resized = cv2.resize(frame, (640, 360))

    # Convert the frame to grayscale
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)

    # Perform template matching
    result = cv2.matchTemplate(frame_gray, template_gray, method)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

    # For TM_CCOEFF_NORMED, the best match is the maximum value
    top_left = max_loc
    bottom_right = (top_left[0] + w, top_left[1] + h)

    # Extract the ROI from the frame
    roi_gray = frame_gray[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]

    # Convert ROI to binary
    _, roi_binary = cv2.threshold(roi_gray, 50, 255, cv2.THRESH_BINARY)

    # Find contours in the binary image
    contours, _ = cv2.findContours(roi_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Find the largest contour
        largest_contour = max(contours, key=cv2.contourArea)

        # Calculate the centroid of the largest contour
        moments = cv2.moments(largest_contour)
        if moments["m00"] != 0:
            cx = int(moments["m10"] / moments["m00"])
            cy = int(moments["m01"] / moments["m00"])
            # Draw the centroid
            cv2.circle(frame_resized, (top_left[0] + cx, top_left[1] + cy), 5, (0, 0, 255), -1)
        else:
            cx, cy = 0, 0

    # Draw a rectangle around the detected object
    cv2.rectangle(frame_resized, top_left, bottom_right, (255, 0, 0), 2)

    # Display the result
    cv2.imshow('Tracking with Centroid', frame_resized)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
